In [1]:
# !pip install focal-loss
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, Input, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
import shutil
from tensorflow.keras import backend as K
from tensorflow.keras import layers, models, regularizers
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers.legacy import Adam
from focal_loss import BinaryFocalLoss
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.losses import BinaryFocalCrossentropy

In [2]:
# glowny folder
base_dir = '/Users/milenabiernacka/Desktop/studia/DS/semestr2/PD-magisterka/Mushroom_dataset/cnn/'
# polaczenie glownego folderu i podfolderow z edible i poisonous

edible_dir = os.path.join(base_dir, 'Edible')
poisonous_dir = os.path.join(base_dir, 'Poisonous')


train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
    )

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Assuming base_dir is the path to the dataset directory
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=1,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

Found 1399 images belonging to 2 classes.
Found 300 images belonging to 2 classes.
Found 301 images belonging to 2 classes.


In [3]:
model = Sequential([
    Input(shape=(150, 150, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Use linear activation for Hinge loss
])


In [4]:
def seesaw_loss_with_penalty(y_true, y_pred, penalty=0.5):
    """
    A simplified conceptual version of seesaw loss for binary classification.
    This function penalizes the loss for the majority class and reduces the penalty
    for the minority class, attempting to balance the seesaw between classes.

    Args:
    - y_true: True labels.
    - y_pred: Predicted labels.
    - penalty: The penalty factor for the majority class. Lower values reduce the loss
               contribution from the majority class.

    Returns:
    - A modified binary cross-entropy loss that incorporates class balancing.
    """
    # Standard binary cross-entropy
    bce = K.binary_crossentropy(y_true, y_pred)

    # Determine weights for each sample based on its true label
    weights = K.abs(y_true - penalty)  # Reduce weight for majority class (assumed to be 1)

    # Apply seesaw penalty
    weighted_bce = weights * bce

    return K.mean(weighted_bce)

In [5]:
model.compile(optimizer=Adam(learning_rate=1e-4), loss=lambda y_true, y_pred: seesaw_loss_with_penalty(y_true, y_pred, penalty=0.8), metrics=['accuracy'])

In [6]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/20
1399/1399 [==============================] - 57s 40ms/step - loss: 0.1770 - accuracy: 0.7362 - val_loss: 0.1479 - val_accuracy: 0.7326
Epoch 2/20
1399/1399 [==============================] - 53s 38ms/step - loss: 0.1392 - accuracy: 0.7412 - val_loss: 0.1297 - val_accuracy: 0.7431
Epoch 3/20
1399/1399 [==============================] - 57s 40ms/step - loss: 0.1188 - accuracy: 0.7777 - val_loss: 0.1182 - val_accuracy: 0.7569
Epoch 4/20
1399/1399 [==============================] - 60s 43ms/step - loss: 0.0982 - accuracy: 0.8392 - val_loss: 0.0760 - val_accuracy: 0.9132
Epoch 5/20
1399/1399 [==============================] - 58s 41ms/step - loss: 0.0831 - accuracy: 0.8721 - val_loss: 0.0654 - val_accuracy: 0.8993
Epoch 6/20
1399/1399 [==============================] - 59s 42ms/step - loss: 0.0696 - accuracy: 0.9021 - val_loss: 0.0557 - val_accuracy: 0.9306
Epoch 7/20
1399/1399 [==============================] - 61s 43ms/step - loss: 0.0637 - accuracy: 0.9056 - val_loss: 0.0447 -

In [7]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

9/9 [==============================] - 127s 16s/step - loss: 0.0198 - accuracy: 0.9792
Test Loss: 0.01980467326939106, Test Accuracy: 0.9791666865348816


In [10]:
# model.compile(optimizer=Adam(learning_rate=1e-4), loss=lambda y_true, y_pred: seesaw_loss_with_penalty(y_true, y_pred, penalty=0.5), metrics=['accuracy'])

In [ ]:
# history = model.fit(
#     train_generator,
#     steps_per_epoch=train_generator.samples // train_generator.batch_size,
#     epochs=20,
#     validation_data=validation_generator,
#     validation_steps=validation_generator.samples // validation_generator.batch_size
# )

In [ ]:
# test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
# print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

### MobileNetV2

In [8]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=1,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

Found 1399 images belonging to 2 classes.
Found 300 images belonging to 2 classes.
Found 301 images belonging to 2 classes.


In [9]:
# Wykorzystanie pretrenowanych sieci
### MobileNetV2
img_height = img_width = 128

# Ładowanie MobileNetV2 bez górnej warstwy (top)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Zamrożenie wagi przetrenowanego modelu
base_model.trainable = False

# Dodanie warstw na wierzchu
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
# To jest tylko przykład, można tutaj dostosować liczbę neuronów i warstw
output = Dense(1, activation='sigmoid')(x)  # Przykładowa warstwa wyjściowa dla klasyfikacji binarnej

# Skompletowanie nowego modelu
model = Model(inputs=base_model.input, outputs=output)

# Kompilacja modelu z niestandardową funkcją straty i optymalizatorem
model.compile(optimizer=Adam(lr=0.0001), loss=lambda y_true, y_pred: seesaw_loss_with_penalty(y_true, y_pred, penalty=0.8), metrics=['accuracy'])
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/20


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


1399/1399 [==============================] - 21s 14ms/step - loss: 0.0558 - accuracy: 0.9214 - val_loss: 0.0180 - val_accuracy: 0.9722
Epoch 2/20
1399/1399 [==============================] - 16s 11ms/step - loss: 0.0252 - accuracy: 0.9671 - val_loss: 0.0158 - val_accuracy: 0.9896
Epoch 3/20
1399/1399 [==============================] - 15s 11ms/step - loss: 0.0166 - accuracy: 0.9793 - val_loss: 0.0172 - val_accuracy: 0.9896
Epoch 4/20
1399/1399 [==============================] - 16s 12ms/step - loss: 0.0095 - accuracy: 0.9878 - val_loss: 0.0148 - val_accuracy: 0.9931
Epoch 5/20
1399/1399 [==============================] - 18s 13ms/step - loss: 0.0145 - accuracy: 0.9836 - val_loss: 0.0112 - val_accuracy: 0.9757
Epoch 6/20
1399/1399 [==============================] - 19s 14ms/step - loss: 0.0169 - accuracy: 0.9814 - val_loss: 0.0082 - val_accuracy: 0.9931
Epoch 7/20
1399/1399 [==============================] - 19s 13ms/step - loss: 0.0120 - accuracy: 0.9857 - val_loss: 0.0087 - val_accura

In [10]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

9/9 [==============================] - 1s 96ms/step - loss: 0.0301 - accuracy: 0.9826
Test Loss: 0.030124111101031303, Test Accuracy: 0.9826388955116272
